In [2]:
import numpy as np
import matplotlib.pyplot as plt
from image_graph_util import img_conversion_v7
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from emnist import extract_training_samples,extract_test_samples
from emnist import list_datasets
from sklearn.utils import shuffle
import cv2
import glob

In [3]:
img=img_conversion_v7.image_convert()


patch=2


In [4]:
ds, info = tfds.load('CatsVsDogs', split='train', shuffle_files=True, with_info=True)

2023-07-25 16:39:06.381850: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/user/home/nvekariy/.local/lib/python3.9/site-packages/cv2/../../lib64:/cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-07-25 16:39:06.381881: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-25 16:39:06.381902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0187): /proc/driver/nvidia/version does not exist
2023-07-25 16:39:06.382433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AV

In [5]:
train_images = []
train_labels = []

for example in ds:
    train_images.append(example['image'].numpy())
    train_labels.append(example['label'].numpy())

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


In [6]:
print(len(train_images))
print(len(train_labels))

23262
23262


In [7]:
images=np.array([cv2.resize(i,(100,100), interpolation= cv2.INTER_CUBIC) for i in train_images])


In [8]:
images = [i/255 for i in images]
labels = np.array(train_labels)

In [9]:
images=np.array(images,dtype="float16")

In [10]:
images.shape

(23262, 100, 100, 3)

In [11]:
train,test,train_lab,test_lab=train_test_split(images,labels,test_size=0.30,random_state=42)


In [12]:
print(train.shape)
print(test.shape)
print(train_lab.shape)
print(test_lab.shape)

(16283, 100, 100, 3)
(6979, 100, 100, 3)
(16283,)
(6979,)


In [13]:
regular_train=train
reg_train_label=train_lab

In [14]:
regular_val,regular_test,reg_val_label,reg_test_label=train_test_split(test,test_lab,test_size=0.50,random_state=42)


In [15]:
print(regular_train.shape)
print(regular_val.shape)
print(regular_test.shape)


(16283, 100, 100, 3)
(3489, 100, 100, 3)
(3490, 100, 100, 3)


In [16]:
from collections import Counter
#print((Counter(y_val)))
print(np.sort(list(Counter(reg_train_label).keys())))
classes=len(np.sort(list(Counter(reg_train_label).keys())))

[0 1]


# Gershgorien image analysis

In [17]:
import networkx as nx



def compute_lap_to_gersh(df_in): #compute_lap_to_gersh
    adj_ary=np.ones(reg_train_patch.shape[-2]*reg_train_patch.shape[-1])-np.eye(reg_train_patch.shape[-2]*reg_train_patch.shape[-1])

    gersh_c=np.tile(np.sum(adj_ary,axis=0),(reg_train_patch.shape[1],reg_train_patch.shape[2]))

    edge=np.array(np.where(adj_ary==1))
    ed0=edge[0]
    ed1=edge[1]
    no_img,no_ch, no_patch,patch_x,patch_y=df_in.shape
    temp=[]
    for i in range(no_img):
        i_ary = np.eye(patch_x*patch_y)
        i_ary4d=i_ary[np.newaxis,np.newaxis,:,:]
        weighted_adj=np.zeros((no_ch, no_patch,patch_x*patch_y,patch_x*patch_y))
        df=df_in[i].reshape(no_ch,no_patch,-1)
        pixel1=df[:,:,ed0]
        pixel2=df[:,:,ed1]  
        edu_distance=np.sqrt((pixel1-pixel2)**2)+ 10**(-8)
        weighted_adj[:,:,ed0,ed1]=edu_distance
        sum_r=np.sum(weighted_adj,axis=2)
        gersh_r=sum_r.reshape(no_ch,-1)
        gersh_total=np.vstack((gersh_r[0],gersh_c[0],gersh_r[1],gersh_c[1],gersh_r[2],gersh_c[2]))
        gersh_total=gersh_total.reshape(no_ch,2,gersh_c.shape[-1])
        temp.append(gersh_total)
    return temp



In [18]:
reg_train_patch=img.to_patcharray(regular_train[:],patch_size=(patch,patch))
reg_val_patch=img.to_patcharray(regular_val[:],patch_size=(patch,patch))
reg_test_patch=img.to_patcharray(regular_test[:],patch_size=(patch,patch))



In [19]:
print(reg_train_patch.shape)
print(reg_val_patch.shape)
print(reg_test_patch.shape)

(16283, 3, 2500, 2, 2)
(3489, 3, 2500, 2, 2)
(3490, 3, 2500, 2, 2)


In [20]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


Number of CPU cores: 48


In [21]:
train_patch1=np.array_split(reg_train_patch[:], num_cores)
val_patch1=np.array_split(reg_val_patch[:], num_cores)
test_patch1=np.array_split(reg_test_patch[:], num_cores)

In [22]:
print(len(train_patch1))
print(len(val_patch1))
print(len(test_patch1))

48
48
48


In [23]:
with Pool(processes=num_cores) as pool:
        # Define a list of numbers
    train_p = train_patch1
    
        # Apply the square function to each number using the pool
    g_train=pool.map(compute_lap_to_gersh,train_p)
    pool.close()
    pool.join()

    
        #results = pool.map(calculate_visibility, numbers)


In [24]:
temp1=[]
for submatrix in g_train:
    temp1.extend(submatrix)


In [25]:
gersh_train= np.array([np.asarray(submatrix) for submatrix in temp1])


In [26]:
with Pool(processes=num_cores) as pool1:
        # Define a list of numbers
    val_p = val_patch1
    
        # Apply the square function to each number using the pool
    g_val=pool1.map(compute_lap_to_gersh,val_patch1)    
    pool1.close()
    pool1.join()



In [27]:
temp2=[]
for submatrix in g_val:
    temp2.extend(submatrix)


In [28]:
gersh_val= np.array([np.asarray(submatrix) for submatrix in temp2])


In [29]:
with Pool(processes=num_cores) as pool2:
        # Define a list of numbers
    test_p = test_patch1
    
        # Apply the square function to each number using the pool
    g_test=pool2.map(compute_lap_to_gersh,test_patch1) 
    pool2.close()
    pool2.join()

In [30]:
temp3=[]
for submatrix in g_test:
    temp3.extend(submatrix)


In [31]:
gersh_test= np.array([np.asarray(submatrix) for submatrix in temp3])


gersh_train=np.array(img1.to_gridlaparray(train_patch))
gersh_val=np.array(img1.to_gridlaparray(val_patch))
gersh_test=np.array(img1.to_gridlaparray(test_patch))

In [32]:
print(gersh_train.shape)
print(gersh_val.shape)
print(gersh_test.shape)

(16283, 3, 2, 10000)
(3489, 3, 2, 10000)
(3490, 3, 2, 10000)


In [33]:
rgersh_train=gersh_train.reshape(reg_train_patch.shape[0],2,reg_train_patch.shape[-3]*reg_train_patch.shape[-2]*reg_train_patch.shape[-1],reg_train_patch.shape[1])
rgersh_val=gersh_val.reshape(reg_val_patch.shape[0],2,reg_train_patch.shape[-3]*reg_train_patch.shape[-2]*reg_train_patch.shape[-1],reg_train_patch.shape[1])
rgersh_test=gersh_test.reshape(reg_test_patch.shape[0],2,reg_train_patch.shape[-3]*reg_train_patch.shape[-2]*reg_train_patch.shape[-1],reg_train_patch.shape[1])


In [34]:
print(rgersh_train.shape)
print(rgersh_val.shape)
print(rgersh_test.shape)

(16283, 2, 10000, 3)
(3489, 2, 10000, 3)
(3490, 2, 10000, 3)


# Gershgorien 2D CNN

In [35]:
input_shape = (rgersh_train.shape[1],rgersh_train.shape[2],rgersh_train.shape[3])
num_classes = classes
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv2D(filters= 32, kernel_size=(1,3), activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.AveragePooling2D(pool_size=(1, 2),strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(1,3),padding='same',activation='relu'))
model3.add(tf.keras.layers.AveragePooling2D(pool_size=(1, 2),strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(1,3),padding='same',activation='relu'))
model3.add(tf.keras.layers.AveragePooling2D(pool_size=(1, 2),strides=None,padding='valid'))

model3.add(tf.keras.layers.Flatten())
model3.add(tf.keras.layers.Dense(512, activation='relu'))

model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(512, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))


In [36]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2, 10000, 32)      320       
                                                                 
 average_pooling2d (AverageP  (None, 2, 5000, 32)      0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 5000, 32)       3104      
                                                                 
 average_pooling2d_1 (Averag  (None, 2, 2500, 32)      0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2500, 32)       3104      
                                                                 
 average_pooling2d_2 (Averag  (None, 2, 1250, 32)      0

In [37]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [38]:
history_log=model3.fit(rgersh_train, reg_train_label, epochs=10,verbose=1,validation_data=(rgersh_val, reg_val_label))


Epoch 1/10
509/509 [==============================] - 50s 96ms/step - loss: 0.7804 - acc: 0.6198 - val_loss: 0.5736 - val_acc: 0.7013
Epoch 2/10
509/509 [==============================] - 48s 94ms/step - loss: 0.5631 - acc: 0.7040 - val_loss: 0.5447 - val_acc: 0.7231
Epoch 3/10
509/509 [==============================] - 48s 94ms/step - loss: 0.4829 - acc: 0.7642 - val_loss: 0.6133 - val_acc: 0.7134
Epoch 4/10
509/509 [==============================] - 48s 94ms/step - loss: 0.3604 - acc: 0.8351 - val_loss: 0.6262 - val_acc: 0.7091
Epoch 5/10
509/509 [==============================] - 48s 94ms/step - loss: 0.1940 - acc: 0.9229 - val_loss: 0.9040 - val_acc: 0.6993
Epoch 6/10
509/509 [==============================] - 48s 94ms/step - loss: 0.0769 - acc: 0.9740 - val_loss: 1.4304 - val_acc: 0.7048
Epoch 7/10
509/509 [==============================] - 48s 95ms/step - loss: 0.0448 - acc: 0.9845 - val_loss: 1.5013 - val_acc: 0.6936
Epoch 8/10
509/509 [==============================] - 48s 94ms

In [39]:
loss, accuracy = model3.evaluate(rgersh_test, reg_test_label)
print(f' Model loss on the test set: {loss}')
print(f' Model accuracy on the test set: {100*accuracy}')

110/110 [==============================] - 2s 15ms/step - loss: 1.6138 - acc: 0.6968
 Model loss on the test set: 1.6137584447860718
 Model accuracy on the test set: 69.68481540679932


In [40]:
predictions = model3.predict(rgersh_test)
pred_arg = np.argmax(predictions, axis=1)

110/110 [==============================] - 2s 15ms/step


In [41]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(reg_test_label,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[1209  541]
 [ 517 1223]]


In [42]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [43]:
print(accuracy_score(reg_test_label, pred_arg)*100)
print(f1_score(reg_test_label,pred_arg,average='macro')*100)
print(recall_score(reg_test_label, pred_arg, average="weighted")*100)
print(precision_score(reg_test_label, pred_arg, average="weighted")*100)

69.68481375358166
69.68432591915338
69.68481375358166
69.6897326132889
